In [ ]:
import os
import glob
import subprocess
import numpy as np
import clima_anom as ca
from subprocess import Popen, PIPE

import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

from os.path import exists as file_exists

FONT: http://ftp.cptec.inpe.br/goes/goes16/glm/

In [ ]:
def get_subdirectories(BASE_URL):
    res = requests.get(BASE_URL)
    soup = BeautifulSoup(res.text, 'html.parser')

    rows = soup.findAll('tr')

    years = []
    for row in rows[3:-1]:
        row = row.find('a')
        years.append(row.attrs['href'])

    return years

In [ ]:
def get_url_files(url):
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    rows = soup.findAll('tr')
    
    urls = list()

    for row in rows[3:-1]:
        row  = row.find('a')
        try:
            link = row.attrs['href']
            urls.append(os.path.join(url, link))
        except:
            pass
    return urls

In [ ]:
def download_file(url_file,file_out):
    res = requests.get(url_file, stream=True)
    total_size = int(res.headers['content-length'])

    with open(file_out, 'wb') as handle:
        for data in tqdm(iterable=res.iter_content(chunk_size=1024), total=total_size/1024, unit='KB'):
            handle.write(data)
    # print("Done!")

In [ ]:
year = '2018'
month = '09'

num_dias, _, _, _ = ca.days_number(year,month)
print(f'Days number: {num_dias}')

In [ ]:
dir_base = '/mnt/Data/GLM/Data/'
if os.path.isdir(dir_base):
    print(f'{dir_base} exists')
else:
    os.mkdir(dir_base)
    print(f'{dir_base} created')

dir_year = dir_base + str(year)
if os.path.isdir(dir_year):
    print(f'{dir_year} exists')
else:
    os.mkdir(dir_year)
    print(f'{dir_year} created')

dir_out = dir_year + '/' + str(month) + '/'
if os.path.isdir(dir_out):
    print(f'{dir_out} exists')
else:
    os.mkdir(dir_out)
    print(f'{dir_out} created')

for dd in range(num_dias):
    day = str(dd+1).zfill(2)
    if os.path.isdir(dir_out + day):
        continue
    else:
        os.mkdir(dir_out + day)

In [ ]:
BASE_URL = 'http://ftp.cptec.inpe.br/goes/goes16/glm'

for dd in range(num_dias):
    day = str(dd + 1).zfill(2)

    print(f'Day: {day}')
    dir_out = '/mnt/Data/GLM/Data/' + year + '/' + month + '/' + day

    url = BASE_URL + '/' + year + '/' + month + '/' + day
    urls = get_url_files(url)

    for n in urls:

        file_name = n.split('/')[-1]

        if not os.path.exists(dir_out+'/'+file_name):
            download_file(n,dir_out+'/'+file_name)
        else:
            print("   File Exists")

    print()